# INSERT INTO DB

In [1]:
import pandas as pd
import numpy as np

import warnings
import os
warnings.filterwarnings(action='ignore')

In [2]:
reviews = pd.read_json('./prc_data/reviews_prc_3.json')

In [16]:
import sqlite3
conn = sqlite3.connect('./apparel.db')
cur = conn.cursor()

### item table

In [ ]:
items = pd.read_json('./items_prc.json')
items.info()

In [8]:
# 데이터 입력에 필요한 형태로 변환
items = items[['item_id', 'cat', 'name', 'price', 'img', 'link']]
items.item_id = items.item_id.astype(str)

In [9]:
for row in items.itertuples(index=False, name=None):
    cur.execute('''
                INSERT INTO item VALUES (?, ?, ?, ?, ?, ?); ''', row)
    conn.commit()

### reviews => user, review로 나누기

In [ ]:
reviews.head()

In [7]:
reviews.replace({np.NaN: None}, inplace=True)

In [8]:
# user, review 정보 나누기
user = reviews[['아이디', '키', '몸무게', '평소사이즈']]
review = reviews[['아이디', '품번', '별점', '리뷰', '사이즈', '색상']]

#### user Table

In [ ]:
for col in ['키', '몸무게', '평소사이즈']:
    print(col)
    print(user[col].unique())

In [ ]:
user_dup = user.loc[user['아이디'].duplicated()].index
user.drop(user_dup, axis=0, inplace=True)
user.reset_index(drop=True, inplace=True)
user.info()


In [11]:
# 아이디 중복 확인 후 데이터 입력
for row in user.itertuples(name=None, index=False):
    # cur.execute(f'''
    #             INSERT INTO user VALUES(?, ?, ?, ?)
    #             SELECT * FROM user
    #             WHERE NOT EXISTS
    #             (SELECT 1 FROM USER
    #             WHERE id == {row[0]});''', row)
    cur.execute('''
                INSERT OR IGNORE
                INTO user VALUES(?,?,?,?);''', row) # 아이디 중복 제외
    
    conn.commit()

In [ ]:
cur.execute('''
            SELECT COUNT(*) FROM user;''')
print(cur.fetchone())

#### review Table

In [ ]:
review = review[['아이디', '품번', '별점', '리뷰', '사이즈', '색상']]
review_dup = review.loc[review['리뷰'].duplicated()==True].index
review.drop(review_dup, axis=0, inplace=True)
review.reset_index(drop=True, inplace=True)
len(review)

In [ ]:
review.duplicated().sum()

In [15]:
# '''
# review table 줄바꿈 제거 필요
# '''
# cur.execute('''
#             UPDATE review
#             SET comment = replace(comment, '\n', ' ');''')
# conn.commit()

In [15]:
for row in review.itertuples(name=None, index=True):
    cur.execute('''
                INSERT OR IGNORE
                INTO REVIEW VALUES(?,?,?,?,?,?,?);''', row)
    conn.commit()

### size Table

In [17]:
# cur.execute('''
#             CREATE TABLE size(
#                 id VARCHAR(1) NOT NULL,
#                 size VARCHAR(10) NOT NULL,
#                 PRIMARY KEY(id));''')
size_id = ['1', '2', '3', '4', '5', 'F']
size = ['XS-S','M', 'L', 'XL', '2XL-3XL', 'F']
s_dict = dict(zip(size_id, size))
for row in s_dict.items():
    cur.execute(f'''
                INSERT INTO size VALUES (?,?);''', row)
    conn.commit()

## 연결 해제

In [18]:
cur.close()
conn.close()